# The Workspace

The workspace pattern provides a shared, persistent file system where agents externalize artifacts too large for the context window. Agents see sandbox paths (`/workspace/...`) while actual files are stored in isolated directories per user and session.

In [ ]:
from pathlib import Path
import tempfile
import json

from agentic_patterns.core.workspace import (
    configure_workspace, DefaultIdentityExtractor,
    container_to_host_path, write_to_workspace, read_from_workspace, list_workspace_files,
)
from agentic_patterns.core.agents import get_agent, run_agent

## Server Startup Configuration

At server startup, configure the workspace with a base directory and an identity extractor. The identity extractor pulls user/session IDs from the HTTP request context (JWT claims, session cookies, etc.).

In [ ]:
base_dir = Path(tempfile.mkdtemp())

# In production, implement IdentityExtractor to read from JWT/session
identity_extractor = DefaultIdentityExtractor(user_id="alice", session_id="session_001")

configure_workspace(WorkspaceConfig(
    base_dir=base_dir,
    identity_extractor=identity_extractor,
))

print(f"Workspace base: {base_dir}")

## Path Translation

Tools receive `ctx` (the request context) from the framework. Helper functions convert between sandbox paths (what agents see) and host paths (actual filesystem).

In [ ]:
# ctx is passed by the framework - for demo we use None (DefaultIdentityExtractor ignores it)
ctx = None

sandbox_path = "/workspace/reports/analysis.json"
host_path = container_to_host_path(sandbox_path, ctx)

print(f"Agent sees:    {sandbox_path}")
print(f"Actual file:   {host_path}")

## Tool Pattern: Write Large Output, Return Summary

When a tool produces large output, it writes the full result to the workspace and returns a concise summary with the file path. The agent's context stays small.

In [ ]:
def analyze_dataset(query: str, ctx) -> str:
    """Analyze data and save results to workspace."""
    result = {
        "query": query,
        "row_count": 50000,
        "statistics": {"mean": 42.5, "std": 12.3, "min": 0.1, "max": 99.8},
        "data": [{"id": i, "value": i * 0.1} for i in range(1000)],
    }
    
    output_path = "/workspace/analysis/result.json"
    write_to_workspace(output_path, json.dumps(result, indent=2), ctx)
    
    return f"""Analysis complete. Rows: {result['row_count']}, Mean: {result['statistics']['mean']}
Full results: {output_path}"""

In [ ]:
summary = analyze_dataset("SELECT * FROM metrics", ctx)
print(summary)

# Verify file exists on disk
host_path = container_to_host_path("/workspace/analysis/result.json", ctx)
print(f"\nFile size: {host_path.stat().st_size} bytes")

## Agent with Workspace Tools

Tools receive `ctx` from the framework and use workspace helpers. The framework injects `ctx` automatically - tools just declare it as a parameter.

In [ ]:
def search_data(query: str, ctx) -> str:
    """Search dataset and save results to workspace."""
    matches = [{"id": i, "name": f"item_{i}", "score": 0.9 - i*0.01} for i in range(500)]
    
    output_path = "/workspace/search_results.json"
    write_to_workspace(output_path, json.dumps(matches), ctx)
    
    return f"Found {len(matches)} matches. Top 3: {matches[:3]}. Full results: {output_path}"


def read_file(path: str, ctx) -> str:
    """Read a file from the workspace."""
    return read_from_workspace(path, ctx)

In [ ]:
agent = get_agent(tools=[search_data, read_file])

prompt = "Search for sensor data and tell me how many results were found."
agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\nAnswer: {agent_run.result.output}")

In [ ]:
# Full data persists in workspace
print("Files in workspace:")
for f in list_workspace_files("*.json", ctx):
    host_path = container_to_host_path(f, ctx)
    print(f"  {f} ({host_path.stat().st_size} bytes)")

## User Isolation

Each user/session gets an isolated directory. The identity extractor determines which workspace to use based on the request context.

In [ ]:
# Reconfigure with different user
configure_workspace(WorkspaceConfig(
    base_dir=base_dir,
    identity_extractor=DefaultIdentityExtractor(user_id="bob", session_id="session_001"),
))

write_to_workspace("/workspace/secret.txt", "Bob's private data", ctx)

# Bob's file is in a different directory
bob_path = container_to_host_path("/workspace/secret.txt", ctx)
print(f"Bob's file: {bob_path}")
print(f"Alice's file would be: {base_dir}/alice/session_001/secret.txt")

## Security: Path Traversal Prevention

In [ ]:
from agentic_patterns.core.workspace import WorkspaceError

try:
    container_to_host_path("/workspace/../../../etc/passwd", ctx)
except WorkspaceError as e:
    print(f"Blocked: {e}")

try:
    container_to_host_path("/etc/passwd", ctx)
except WorkspaceError as e:
    print(f"Blocked: {e}")